In [18]:
#import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.tsaplots import plot_acf
import os

In [20]:
def generate_data(T, alpha, beta, sigma, alpha_y, beta_y, gamma, theta, sigma_y, mu):
    df = pd.DataFrame(index=range(T), columns=['x', 'y', 'epsilon_x', 'epsilon_y'])
    x1 = alpha/(1-beta) # define initial value for time series x
    y1 = 0
    epsilon_x = sigma*np.random.randn(T) + mu # generate a vector of T random normal
    epsilon_y = sigma_y*np.random.randn(T) # generate a vector of T random normal
    
    df.iloc[0,:] = [x1, y1, np.nan, np.nan] # initialize x and y
    
    x = np.zeros(T)
    y = np.zeros(T)
    x[0] = x1
    y[0] = y1
    
    for i in range(0, T-1):
        x[i+1] = alpha + beta * x[i] + epsilon_x[i+1] # generate X(t) recursively
            # Xt = α + βXt-1 + εt
        y[i+1] = alpha_y + beta_y * y[i] + gamma * x[i+1] + epsilon_y[i+1] + theta * epsilon_y[i] # generate Y(t) recursively
            # Yt = α + βYt-1 + γXt + εt + θεt-1
        df.iloc[i+1,0] = x[i+1]
        df.iloc[i+1,1] = y[i+1]
        df.iloc[i+1,2] = epsilon_x[i+1] # associate error with X(t)
        df.iloc[i+1,3] = epsilon_y[i+1] # associate error with Y(t)
    
    return df, x, y, epsilon_x, epsilon_y

In [21]:
df, x, y, epsilon_x, epsilon_y = generate_data(1000, 1, 0.9, 0.1, 1, 0.9, 0.9, 0.5, 0.1, 0)
new_df = df.drop(0)

In [34]:
import numpy as np
from scipy.stats import norm

def quantile_regression_gls(y, X, tau):
    n, p = X.shape
    nq = len(tau)
    residuals = np.zeros((n, nq))
    beta = np.zeros((nq, p+1))
    weights = np.zeros((n, nq))
    Xw = np.zeros((n, nq*p+p+1))

    for i in range(nq):
        q = tau[i]
        for j in range(n):
            residuals[j, i] = y[j] - np.dot(X[j,:], beta[i,:])
            weights[j,i] = norm.pdf(residuals[j,i]/weights[j,i])
        Xw[:, i*p+1:(i+1)*p+1] = np.multiply(X, np.tile(weights[:,i],(p,1)).T)
        Xw[:, i*p+1] = np.tile(weights[:,i],(1,1)).T
        Xw[:, -p-1] = 1
        beta[i,:] = np.linalg.inv(Xw.T @ Xw) @ Xw.T @ np.multiply(y, np.tile(weights[:,i],(1,1)).T)

    return beta



In [35]:
y_lag = np.roll(y, 1)
y_lag[0] = 0
epsilon_lag = np.roll(epsilon_y, 1)
epsilon_lag[0] = 0
X = np.column_stack((np.ones_like(x), y_lag, x, epsilon_lag))
tau = [0.1, 0.5, 0.9]
quantile_regression_gls(y, X, tau)

ValueError: shapes (4,) and (5,) not aligned: 4 (dim 0) != 5 (dim 0)